## Twitter sentiment analysis 

- [NLTK documentation](https://www.nltk.org/howto/collocations.html)
- [sentiment analysis tutorial](https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk)
- [Scrape tweets from twitter](https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1)

This script performs the folling functions 

- Import necessary libraries 
- Script to scrape tweets from an account with retweets 
- Script to scrape tweets from an account without retweets 
- Script to scrape tweets from all accounts based on keyword 
- Run data frame through a script that creates a new column with a boolean value if it contains profanity
- Run the dataframe through a profanity filter to remove any profanities (not completed)
- Run script to censor dataframes for profanity (not completed)


In [1]:
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords') #Dont need this unless stopwords need to be updated
import string
#This package will allow us to remove punctuation
from nltk.tokenize import RegexpTokenizer
#import regex
#import twitter scraper 
import tweepy
#Import Json to allow us to get an organize the info from twitter
import json
#Import datetime to use in our dataframe 
import datetime
#import pandas to create and manipulate dataframes 
import pandas as pd
#import joblib
from profanity_check import predict, predict_prob
#Import profanity Filter 
from profanity_filter import ProfanityFilter

#### Twitter API keys
- consumer_key = "KasxVpXExYm4hVdMo0ptDcEhb"
- consumer_secret = "V9mCeYXPDTaPVHCscHcp3g2G53FFBCykfMzF92CTur4hRcuhjr"
- access_token = "152439706-y5Titlw7pDkXzX8ooaBRR3UQW1UnUkmSUOSwRvZU"
- access_token_secret = "83kTOT7idOvGCA75jtqq36voUAHVEBZMRKuHRRHk9lbYA"


In [2]:
#This script will print tweets from my homepage - it is unstructured

#Add your credentials here
twitter_keys = {
        'consumer_key':        'KasxVpXExYm4hVdMo0ptDcEhb',
        'consumer_secret':     'V9mCeYXPDTaPVHCscHcp3g2G53FFBCykfMzF92CTur4hRcuhjr',
        'access_token_key':    '152439706-y5Titlw7pDkXzX8ooaBRR3UQW1UnUkmSUOSwRvZU',
        'access_token_secret': '83kTOT7idOvGCA75jtqq36voUAHVEBZMRKuHRRHk9lbYA'
    }

#Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth)

#Make call on home timeline, print each tweets text
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

Uber ex-security chief accused of hacking coverup must face fraud charges, judge rules https://t.co/xldf1xdKPF https://t.co/WD5kjd1U8D
From @Breakingviews: Robinhood’s descent has been as dramatic as its rise, but the company still has millions of us… https://t.co/udk5GvJamA
CORIOLANUS. My gracious silence, hail!
RT @joshgnosis: Uhh if they're shocked about the display of homosexuality at a performing arts school I have questions. https://t.co/Z8gZvF…
RT @GovWhitmer: This Governor is proud of Lake Superior, which has really earned its name.
RT @iamharaldur: A tourist asked me recently why there were so many people in wheelchairs in Reykjavik.

I told him his country had them to…
Justice Samuel Alito argued that “Roe ‘inflamed’ a national issue that has remained bitterly divisive for the past… https://t.co/Iqre2Fb3s3
The mountains of Papua New Guinea were once home to kangaroos with 'unique teeth' https://t.co/kKD4bUmIzt
Princess Ibini scores her first Matildas goal in draw with Portgua

In [3]:
#This will print all of the metadata from the above tweets
public_tweets[0]

Status(_api=<tweepy.api.API object at 0x7fe8d0ed44c0>, _json={'created_at': 'Wed Jun 29 00:30:25 +0000 2022', 'id': 1541942126312824832, 'id_str': '1541942126312824832', 'text': 'Uber ex-security chief accused of hacking coverup must face fraud charges, judge rules https://t.co/xldf1xdKPF https://t.co/WD5kjd1U8D', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/xldf1xdKPF', 'expanded_url': 'http://reut.rs/3I3iY3A', 'display_url': 'reut.rs/3I3iY3A', 'indices': [87, 110]}], 'media': [{'id': 1541942124362481664, 'id_str': '1541942124362481664', 'indices': [111, 134], 'media_url': 'http://pbs.twimg.com/media/FWYUMUrWQAAb2XT.jpg', 'media_url_https': 'https://pbs.twimg.com/media/FWYUMUrWQAAb2XT.jpg', 'url': 'https://t.co/WD5kjd1U8D', 'display_url': 'pic.twitter.com/WD5kjd1U8D', 'expanded_url': 'https://twitter.com/Reuters/status/1541942126312824832/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 

In [4]:
status = public_tweets[0]

#convert to string
json_str = json.dumps(status._json)

#deserialise string into python object
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Wed Jun 29 00:30:25 +0000 2022",
    "entities": {
        "hashtags": [],
        "media": [
            {
                "display_url": "pic.twitter.com/WD5kjd1U8D",
                "expanded_url": "https://twitter.com/Reuters/status/1541942126312824832/photo/1",
                "id": 1541942124362481664,
                "id_str": "1541942124362481664",
                "indices": [
                    111,
                    134
                ],
                "media_url": "http://pbs.twimg.com/media/FWYUMUrWQAAb2XT.jpg",
                "media_url_https": "https://pbs.twimg.com/media/FWYUMUrWQAAb2XT.jpg",
                "sizes": {
                    "large": {
                        "h": 381,
                        "resize": "fit",
                        "w": 728
                    },
                    "medium": {
                        "h": 381,
                        "resize": "fit",
           

In [5]:
#include retweets

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        'KasxVpXExYm4hVdMo0ptDcEhb',
        'consumer_secret':     'V9mCeYXPDTaPVHCscHcp3g2G53FFBCykfMzF92CTur4hRcuhjr',
        'access_token_key':    '152439706-y5Titlw7pDkXzX8ooaBRR3UQW1UnUkmSUOSwRvZU',
        'access_token_secret': '83kTOT7idOvGCA75jtqq36voUAHVEBZMRKuHRRHk9lbYA'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="AOC", #Alexandra Ocasio Cortez 
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [6]:
#Remove RETWEETS 
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        'KasxVpXExYm4hVdMo0ptDcEhb',
        'consumer_secret':     'V9mCeYXPDTaPVHCscHcp3g2G53FFBCykfMzF92CTur4hRcuhjr',
        'access_token_key':    '152439706-y5Titlw7pDkXzX8ooaBRR3UQW1UnUkmSUOSwRvZU',
        'access_token_secret': '83kTOT7idOvGCA75jtqq36voUAHVEBZMRKuHRRHk9lbYA'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="MMFlint", #Michael Moore 
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended'
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended')
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [7]:
miner=TweetMiner(result_limit = 20 )

In [8]:
#Print Mined tweets as a dataframe
mined_tweets = miner.mine_user_tweets(user='MMFlint', max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [9]:
#Print the 7th row to see if script is accurate 
mined_tweets_df.loc[[6]]

,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
6,1540880316352577537,Michael Moore,MMFlint,654,This is my theater in Michigan. We’re actually...,2022-06-28 20:32:16.838461,2022-06-26 02:11:10+00:00,5333,[],16001,None,Twitter for iPhone,None,None,None


#### Search tweets for Biden 
https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/

In [42]:
#Key word search Query is not case sensitive 

search_query = "biden -filter:retweets"

In [49]:
# get tweets from the API
tweets = tweepy.Cursor(api.search_tweets,
              q=search_query,
              lang="en",
              since="2022-06-28").items(500)
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


Total Tweets fetched: 500


In [45]:
tweets_copy

[Status(_api=<tweepy.api.API object at 0x7fe8d0ed44c0>, _json={'created_at': 'Wed Jun 29 01:25:36 +0000 2022', 'id': 1541956012537778176, 'id_str': '1541956012537778176', 'text': '@RonnyJacksonTX Hahahaha.  Biden’s fault huh…… who’s the governor of Texas??', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'RonnyJacksonTX', 'name': 'Ronny Jackson', 'id': 1201953403099893760, 'id_str': '1201953403099893760', 'indices': [0, 15]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 1541827228794847234, 'in_reply_to_status_id_str': '1541827228794847234', 'in_reply_to_user_id': 1201953403099893760, 'in_reply_to_user_id_str': '1201953403099893760', 'in_reply_to_screen_name': 'RonnyJacksonTX', 'user': {'id': 2149200037, 'id_str': '2149200037', 'name': 'David Smith', 'screen_name': 'desmith1426', 'l

In [50]:

# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)
# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Jose arber,,None,False,2022-06-29 01:32:18+00:00,Who will Biden blame again. Guess!,None,Twitter for iPhone
1,Thomas Billue,"Fort Worth, TX","Widowed Veteran, Father and former Husband and...",False,2022-06-29 01:32:17+00:00,@ReefSixtyNine69 Joe Biden should have to pay ...,None,Twitter for Android
2,tomoz 🧼🇺🇦,Northern California,College student who occasionally makes maps am...,False,2022-06-29 01:32:16+00:00,Is this all mail-ins? Brooks is outrunning Bid...,None,Twitter for iPhone
3,Newsericks,"Washington, DC & PA",I'm a pragmatic progressive. I oppose bigotry+...,False,2022-06-29 01:32:16+00:00,@MarshaBlackburn Congratulations to the Biden ...,None,Twitter Web App
4,MikeVizyak,,,False,2022-06-29 01:32:16+00:00,@Gian_Pop92 @ImperialDJoha @Justmyopin10n @mos...,None,Twitter for Android


In [51]:
#This function adds a new column into the dataframe above and uses a boolean value to let you know if it contains profanities    

def detect_profanity(df): 
    df['contains_profanity'] = predict(df['text'])
    return df 


In [52]:
tweets_df_prof= detect_profanity(tweets_df)
tweets_df_prof

,user_name,user_location,user_description,user_verified,date,text,hashtags,source,contains_profanity
0,Jose arber,,None,False,2022-06-29 01:32:18+00:00,Who will Biden blame again. Guess!,None,Twitter for iPhone,0
1,Thomas Billue,"Fort Worth, TX","Widowed Veteran, Father and former Husband and...",False,2022-06-29 01:32:17+00:00,@ReefSixtyNine69 Joe Biden should have to pay ...,None,Twitter for Android,0
2,tomoz 🧼🇺🇦,Northern California,College student who occasionally makes maps am...,False,2022-06-29 01:32:16+00:00,Is this all mail-ins? Brooks is outrunning Bid...,None,Twitter for iPhone,0
3,Newsericks,"Washington, DC & PA",I'm a pragmatic progressive. I oppose bigotry+...,False,2022-06-29 01:32:16+00:00,@MarshaBlackburn Congratulations to the Biden ...,None,Twitter Web App,0
4,MikeVizyak,,,False,2022-06-29 01:32:16+00:00,@Gian_Pop92 @ImperialDJoha @Justmyopin10n @mos...,None,Twitter for Android,0
...,...,...,...,...,...,...,...,...,...
495,Greg Cabral ☮️❤️✌️,,I am mine.,False,2022-06-29 01:26:40+00:00,@yourmansnthem @WalserWes @GOPChairwoman More ...,None,Twitter for iPhone,0
496,Hi,"San Antonio, TX",Occasionally tweets about baseball and politic...,False,2022-06-29 01:26:39+00:00,@GregAbbott_TX If you can rationalize that the...,None,Twitter for iPhone,0
497,💖 M34n 9w0r1 ✨,Outerspace,𝐌𝐮𝐬𝐢𝐜𝐢𝐚𝐧 👩🏾‍🎤 “Mean Girl Remix”ᴏᴜᴛ ɴᴏᴡ 📥Deluxx...,False,2022-06-29 01:26:39+00:00,“Feeling Presidential they Joe Biden (Biting) ...,None,Twitter for iPhone,0
498,Fish,,I do molecular neurobiology. I like synaptic p...,False,2022-06-29 01:26:37+00:00,@Imageless18 @Waffle_64 @jon45444945 @POTUS Th...,None,Twitter for Android,1


In [55]:
#Print only profanity tweets to see if this script works 
profanity_tweet = tweets_df_prof[tweets_df_prof['contains_profanity'] == 1]
profanity_tweet

,user_name,user_location,user_description,user_verified,date,text,hashtags,source,contains_profanity
34,Joyce Vann 🇺🇦🌻🇺🇦🌻🇺🇦🌻,"Little Elm, TX",Proud to be a Liberal/Feminist since the 70’s....,False,2022-06-29 01:31:48+00:00,@Jim_Jordan @DonaldJTrumpJr Shut up about Hunt...,None,Twitter for iPhone,1
49,figaro cicero,"Juneau, AK",camel 🐪,False,2022-06-29 01:31:39+00:00,Call me Joe Biden because I DO NOT FUCK W YOU ...,None,Twitter for iPhone,1
93,TwoCents5818,,"Women, Kids, animals, Earth, life, justice, de...",False,2022-06-29 01:31:13+00:00,At this point I'm glad the orange idiot was to...,None,Twitter for Android,1
135,Detlef Schrempf,,violating community guidelines,False,2022-06-29 01:30:43+00:00,In the past 3 years you’ve sent us what? 1 fuc...,None,Twitter for iPhone,1
141,Ben stedman,,,False,2022-06-29 01:30:40+00:00,@Blonndie313 @PalmerReport @PeterAlexander She...,None,Twitter Web App,1
160,FluxQpacitor,"Magacountry,USA",#maga#Trump2024#epsteindidntkillhimself#maxwel...,False,2022-06-29 01:30:30+00:00,"And a daily reminder, it's not so much Biden t...",None,Twitter for Android,1
182,TIME TO FIGHT FOR THE CHILDREN!!,TEXAS,TRUMP WON!! FBAP ❤🤍💙\nADVOCATING FOR CHILDREN ...,False,2022-06-29 01:30:19+00:00,"More illegal blood on Biden""s head.\nIMPEACH t...",None,Twitter for Android,1
215,Bob,,Freedom loving American.,False,2022-06-29 01:29:59+00:00,@FoxNews @GregAbbott_TX The Biden administrati...,None,Twitter for iPhone,1
234,Sappy in usa,,"Passionately Love my family, dirt roads, gard...",False,2022-06-29 01:29:43+00:00,@mcnerfball @Redistrict @CookPolitical We don’...,None,Twitter for iPad,1
239,Robert Francis,"Huntington Beach, CA","Father,Husband, American, Veteran, Employed, M...",False,2022-06-29 01:29:40+00:00,@KurtSchlichter Or Joe Biden. Didnt he shit hi...,None,Twitter for iPhone,1


### Profanity Filter 

##### Task - write a script that will censor words but leave tweets in tact - this will help to provide analysis later on. 

https://lubna2004.medium.com/profanity-to-be-or-not-to-be-dd32d53648f7

In [14]:

predict(['CRAP! What a stupid decision!'])

#Crap and Stupid are examples of profanities 
#This is delineated by the boolean value of one 

array([1])

In [15]:
predict(['Lovely Spring Weather!'])
#There are no profanities contained in this

array([0])

In [16]:
#Use profanity Filter - to change and censor profane words 

In [17]:
pf = ProfanityFilter()
pf.censor_char = '@'
pf.censor("Crap! What a stupid decision")

'@@@@! What a stupid decision'

In [31]:
tweets_string = tweets_df['text'] = tweets_df['text'].astype(str)

tweets_string#pf.censor(tweets_string)

0    @POTUS Biden’s Border crisis is the direct cau...
1    Biden’s Energy Agenda is Dependent On CHINA.  ...
2    @GeraldoRivera The Five completely ignored thi...
3    @Ginno41238049 Exactly. They use the hapless B...
4    @CarlaMorotoli @TexasTribune There hasn't been...
5    Hunter Biden Only Had 4 Words to Say to Damnin...
6    @MarshaBlackburn Check at Biden's place.  He l...
7    @Sifill_LDF @JoeHill111 Do you mean the people...
8    @MysterySolvent She reminds me of a horse face...
9    @RedondoStrand @DanRather Why is Biden sitting...
Name: text, dtype: object

In [ ]:
#Search for tweets matching specific keywords 

In [20]:
tweets_df.text

0    @POTUS Biden’s Border crisis is the direct cau...
1    Biden’s Energy Agenda is Dependent On CHINA.  ...
2    @GeraldoRivera The Five completely ignored thi...
3    @Ginno41238049 Exactly. They use the hapless B...
4    @CarlaMorotoli @TexasTribune There hasn't been...
5    Hunter Biden Only Had 4 Words to Say to Damnin...
6    @MarshaBlackburn Check at Biden's place.  He l...
7    @Sifill_LDF @JoeHill111 Do you mean the people...
8    @MysterySolvent She reminds me of a horse face...
9    @RedondoStrand @DanRather Why is Biden sitting...
Name: text, dtype: object

In [ ]:
#This script collects tweets that have the keyword - "Biden or biden"

from tweepy import OAuth1UserHandler, API
auth = OAuth1UserHandler(consumer_key, consumer_secret, access_token_key, access_token_secret)
api = API(auth)
KEYWORDS = "biden OR Biden"
# Basic keyword search with full text in english
tweets = api.search_tweets(KEYWORDS,lang="en",count=20, tweet_mode="extended")
print("Total:", len(tweets))


In [26]:
tweets_string

0    @POTUS Biden’s Border crisis is the direct cau...
1    Biden’s Energy Agenda is Dependent On CHINA.  ...
2    @GeraldoRivera The Five completely ignored thi...
3    @Ginno41238049 Exactly. They use the hapless B...
4    @CarlaMorotoli @TexasTribune There hasn't been...
5    Hunter Biden Only Had 4 Words to Say to Damnin...
6    @MarshaBlackburn Check at Biden's place.  He l...
7    @Sifill_LDF @JoeHill111 Do you mean the people...
8    @MysterySolvent She reminds me of a horse face...
9    @RedondoStrand @DanRather Why is Biden sitting...
Name: text, dtype: object